In [42]:
from recursive_clustering.models.estimator import RecursiveClustering
from sklearn.datasets import make_classification
from sklearn.metrics.pairwise import pairwise_kernels, euclidean_distances
import numpy as np
from sklearn.utils import check_random_state
from scipy.sparse.csgraph import connected_components
from scipy.spatial.distance import pdist,squareform

n_samples = int(100)
n_features = 10
n_classes = 5
random_state = 0
class_sep = 50
X, y = make_classification(n_samples=n_samples, n_features=n_features, n_informative=n_features, n_classes=n_classes,
                           n_repeated=0, n_redundant=0, n_clusters_per_class=1,
                           random_state=random_state, class_sep=class_sep)
X = (X - X.mean(axis=0))/X.std(axis=0)
# model = KernelKMeans()
# model = RecursiveClustering(kernel_kmeans=True, kkmeans_gamma=100, kmeans_verbose=1, kmeans_n_init=1)
# model.fit(X)
# labels = model.labels_
# print(labels)


In [2]:
def kernel(x, y, w, h):
    return np.exp(-np.sum((x - y) ** 2 * w) / h)

In [110]:
lambda_ = 1
h = 0.2
n, p = X.shape
X2 = X.copy()
K_matrix = np.zeros((n, n))
wo = np.ones(p)
D = np.zeros(p)

X1 = X.copy()
X2 = X.copy()
wi = wo.copy()
# Initial 20 iterations
for t in range(20):
    for i in range(n):
        for j in range(i, n):
            K_matrix[i, j] = kernel(X2[i], X2[j], wi, h)
    K_matrix += K_matrix.T - np.diag(K_matrix.diagonal())

    for i in range(n):
        I = np.delete(np.arange(n), i)
        s = np.sum(K_matrix[I, i])
        for l in range(p):
            X1[i,l] = np.sum(X2[I,l]*K_matrix[I,i])
        X1[i] = X1[i]/s

    D = np.sum((X - X1) ** 2, axis=0)
    wi = np.exp(-D / lambda_)
    wi /= np.sum(wi)

    X2 = X1.copy()
    # if t == 1:
    #     raise

X1 = X.copy()
X2 = X.copy()
for t in range(30):
    for i in range(n):
        for j in range(i, n):
            K_matrix[i, j] = kernel(X2[i], X2[j], wi, h)
    K_matrix += K_matrix.T - np.diag(K_matrix.diagonal())

    for i in range(n):
        I = np.delete(np.arange(n), i)
        s = np.sum(K_matrix[I, i])
        for l in range(p):
            X1[i,l] = np.sum(X2[I,l]*K_matrix[I,i])
        X1[i] = X1[i]/s

    D = np.sum((X - X1) ** 2, axis=0)
    wi = np.exp(-D / lambda_)
    wi /= np.sum(wi)

    X2 = X1.copy()

epsa = 1e-10
U = X2.copy()
n = U.shape[0]
A = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        if np.sqrt(np.sum((U[i,:]-U[j,:])**2))<epsa:
            A[i,j]=1
_, labels = connected_components(A, directed=False)

In [78]:
wo = np.ones(p) / p
K = euclidean_distances(X*np.sqrt(wo), squared=True)

In [79]:
K2 = squareform(pdist(X, 'sqeuclidean', w=wo))

In [81]:
np.isclose(K,K2).sum()

np.int64(10000)

In [74]:
K == K2

array([[ True, False,  True, ..., False, False,  True],
       [False,  True,  True, ..., False, False, False],
       [ True, False,  True, ..., False,  True, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False,  True, ..., False,  True, False],
       [ True, False, False, ...,  True,  True,  True]])

In [75]:
K2

array([[0.00000000e+00, 1.61265014e-02, 3.55263363e+01, ...,
        2.36079384e+01, 2.31453189e+01, 2.34518303e+01],
       [1.61265014e-02, 0.00000000e+00, 3.56905209e+01, ...,
        2.41204134e+01, 2.36376214e+01, 2.33982621e+01],
       [3.55263363e+01, 3.56905209e+01, 0.00000000e+00, ...,
        2.53354589e+01, 2.48529855e+01, 2.54805471e+01],
       ...,
       [2.36079384e+01, 2.41204134e+01, 2.53354589e+01, ...,
        0.00000000e+00, 1.64079073e-02, 3.06989341e+01],
       [2.31453189e+01, 2.36376214e+01, 2.48529855e+01, ...,
        1.64079073e-02, 0.00000000e+00, 2.99008996e+01],
       [2.34518303e+01, 2.33982621e+01, 2.54805471e+01, ...,
        3.06989341e+01, 2.99008996e+01, 0.00000000e+00]])

In [76]:
K

array([[0.00000000e+00, 1.61265014e-02, 3.55263363e+01, ...,
        2.36079384e+01, 2.31453189e+01, 2.34518303e+01],
       [1.61265014e-02, 0.00000000e+00, 3.56905209e+01, ...,
        2.41204134e+01, 2.36376214e+01, 2.33982621e+01],
       [3.55263363e+01, 3.56905209e+01, 0.00000000e+00, ...,
        2.53354589e+01, 2.48529855e+01, 2.54805471e+01],
       ...,
       [2.36079384e+01, 2.41204134e+01, 2.53354589e+01, ...,
        0.00000000e+00, 1.64079073e-02, 3.06989341e+01],
       [2.31453189e+01, 2.36376214e+01, 2.48529855e+01, ...,
        1.64079073e-02, 0.00000000e+00, 2.99008996e+01],
       [2.34518303e+01, 2.33982621e+01, 2.54805471e+01, ...,
        3.06989341e+01, 2.99008996e+01, 0.00000000e+00]])

In [121]:
lambda_ = 1
h = 0.2
n, p = X.shape
X2_2 = X.copy()
X1_2 = X.copy()
wo = np.ones(p)
wi_2 = wo.copy()
for t in range(20):
    # K_matrix_2 = X2_2 * np.sqrt(wi_2)
    # K_matrix_2 = euclidean_distances(K_matrix_2, squared=True)
    K_matrix_2 = squareform(pdist(X2_2, 'euclidean', w=wi_2))
    K_matrix_2 = np.exp(-K_matrix_2 / h)
    np.fill_diagonal(K_matrix_2,0)
    s_2_2 = np.sum(K_matrix_2, axis=1, keepdims=True)
    X1_2 = (X2_2.T @ K_matrix_2).T / s_2_2
    D_2 = np.sum((X - X1_2) ** 2, axis=0)
    wi_2 = np.exp(-D_2 / lambda_)
    wi_2 /= np.sum(wi_2)
    X2_2 = X1_2.copy()
    # if t == 1:
    #     raise

X1_2 = X.copy()
X2_2 = X.copy()
for t in range(30):
    # K_matrix_2 = X2_2 * np.sqrt(wi_2)
    # K_matrix_2 = euclidean_distances(K_matrix_2, squared=True)
    K_matrix_2 = squareform(pdist(X2_2, 'euclidean', w=wi_2))
    K_matrix_2 = np.exp(-K_matrix_2 / h)
    np.fill_diagonal(K_matrix_2,0)
    s_2_2 = np.sum(K_matrix_2, axis=1, keepdims=True)
    X1_2 = (X2_2.T @ K_matrix_2).T / s_2_2
    D_2 = np.sum((X - X1_2) ** 2, axis=0)
    wi_2 = np.exp(-D_2 / lambda_)
    wi_2 /= np.sum(wi_2)
    X2_2 = X1_2.copy()

epsa = 1e-10
U_2 = X2_2.copy()
n = U_2.shape[0]
A_2 = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        if np.sqrt(np.sum((U_2[i,:]-U_2[j,:])**2))<epsa:
            A_2[i,j]=1
_, labels_2 = connected_components(A_2, directed=False)

In [122]:
labels

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
      dtype=int32)

In [124]:
labels_2

array([0, 0, 1, 2, 3, 1, 0, 0, 2, 1, 2, 0, 0, 0, 2, 3, 0, 0, 2, 2, 2, 0,
       3, 3, 3, 0, 0, 1, 3, 1, 0, 3, 3, 0, 0, 0, 3, 0, 0, 3, 2, 0, 1, 0,
       1, 3, 0, 0, 1, 0, 1, 2, 0, 3, 3, 0, 1, 0, 2, 0, 3, 0, 1, 2, 3, 0,
       2, 2, 0, 0, 0, 2, 0, 2, 1, 1, 1, 0, 1, 3, 2, 2, 0, 1, 0, 3, 2, 3,
       0, 1, 0, 3, 0, 0, 1, 1, 1, 2, 2, 3], dtype=int32)

In [115]:
X2

array([[ 0.49395223, -0.49763125,  1.2199074 ,  0.28675956, -1.21997813,
        -0.49688302,  0.81462487,  0.81415933,  0.10923769, -0.29115395],
       [ 0.49395244, -0.49763112,  1.21990706,  0.28672817, -1.21997784,
        -0.49688296,  0.81462496,  0.8141593 ,  0.10926892, -0.29112265],
       [ 0.494468  , -0.49543536, -0.81229178, -0.81740797,  0.80740006,
         1.99652268, -1.2234902 , -1.22325408,  1.22873459,  0.82213051],
       [-1.99613297, -0.49652445, -0.82613825,  1.22256517,  0.81817996,
        -0.49667582, -1.22340123, -1.2239364 , -0.80458641, -1.22030001],
       [ 0.51265105,  1.98635879, -0.79925968, -0.81121814,  0.81250093,
        -0.50651013,  0.81724176,  0.81910026, -0.80907238,  0.81344752],
       [ 0.49446791, -0.49543536, -0.81229181, -0.8174079 ,  0.80740009,
         1.99652262, -1.22349023, -1.2232541 ,  1.22873452,  0.82213045],
       [ 0.49395223, -0.49763111,  1.21990723,  0.28675478, -1.21997796,
        -0.49688301,  0.81462486,  0.81415929

In [116]:
X2_2

array([[ 0.49472056, -0.50077781,  1.22224009,  0.2045793 , -1.22244185,
        -0.4969534 ,  0.81525118,  0.8144758 ,  0.19384583, -0.20914898],
       [ 0.49472056, -0.50077781,  1.22224009,  0.2045793 , -1.22244185,
        -0.4969534 ,  0.81525118,  0.8144758 ,  0.19384583, -0.20914898],
       [ 0.47938053, -0.49554194, -0.81290088, -0.80520189,  0.80776708,
         1.98230052, -1.22409571, -1.22381574,  1.21681009,  0.80984984],
       [-1.98178114, -0.4965176 , -0.82655566,  1.21038948,  0.81858944,
        -0.48171937, -1.22385316, -1.22436249, -0.79246262, -1.20807855],
       [ 0.51268425,  1.99011933, -0.80210321, -0.81012431,  0.81541601,
        -0.50682185,  0.8174423 ,  0.81926723, -0.81329765,  0.81231793],
       [ 0.47938053, -0.49554194, -0.81290088, -0.80520189,  0.80776708,
         1.98230052, -1.22409571, -1.22381574,  1.21681009,  0.80984984],
       [ 0.49472056, -0.50077781,  1.22224009,  0.2045793 , -1.22244185,
        -0.4969534 ,  0.81525118,  0.8144758 

In [117]:
np.isclose(X2, X2_2)

array([[False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, 

In [118]:
np.isclose(wi, wi_2)

array([False, False, False,  True, False, False, False, False,  True,
        True])

In [119]:
wi

array([1.35028065e-01, 1.29576451e-01, 1.40618109e-01, 8.13663224e-17,
       1.46608774e-01, 1.42608369e-01, 1.50633236e-01, 1.54926996e-01,
       6.96367355e-17, 9.55764599e-17])

In [120]:
wi_2

array([1.33917517e-01, 1.30162604e-01, 1.41094930e-01, 1.19409647e-19,
       1.47132978e-01, 1.41480098e-01, 1.50952204e-01, 1.55259670e-01,
       1.04381472e-19, 1.45470165e-19])

In [29]:
wi_2.dtype

dtype('float64')

In [29]:
X2 - X2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# for t in range(20):
t = 0
K_matrix = np.exp()
K_matrix = euclidean_distances(X2, squared=True)
K_matrix = np.exp(-K_matrix / h)

In [15]:
distances = euclidean_distances(X)
median_distance = np.median(distances)

In [16]:
median_distance

np.float64(696.7999544696155)

In [17]:
gamma = 1 / (2 * median_distance)

In [18]:
gamma

np.float64(0.0007175660629607617)

In [81]:
gamma = 1 / (2 * median_distance)
K = pairwise_kernels(X, metric='rbf', gamma=gamma)
within_distances = np.zeros(n_classes)
random_state = check_random_state(42)
labels = random_state.randint(n_classes, size=n_samples)
dist_mask = np.zeros((n_samples, n_classes))

In [83]:
for j in range(n_classes):
    mask = labels == j
    sw = np.ones(n_samples)
    denom_mask = sw[mask].sum()
    denomsq_mask = denom_mask * denom_mask
    KK_mask = K[mask][:, mask]  # K[mask, mask] does not work.
    dist_j_mask = np.sum(np.outer(sw[mask], sw[mask]) * KK_mask / denomsq_mask)
    within_distances[j] = dist_j_mask
    dist_mask[:, j] += dist_j_mask
    dist_mask[:, j] -= 2 * np.sum(sw[mask] * K[:, mask], axis=1) / denom_mask

In [84]:
2 * np.sum(sw[mask] * K[:, mask], axis=1) / denom_mask

array([0.00296766, 0.01462357, 0.00355041, 0.01210312, 0.01395799,
       0.00705427, 0.00503069, 0.00347525, 0.00365044, 0.01248924,
       0.00548069, 0.00490935, 0.01307933, 0.00473925, 0.00925257,
       0.00593841, 0.00254588, 0.01494558, 0.00548426, 0.00606662,
       0.00762713, 0.01241508, 0.00831312, 0.00481035, 0.00220032,
       0.00470827, 0.0053419 , 0.00456536, 0.0076601 , 0.00184823,
       0.00427364, 0.0021476 , 0.01429127, 0.00238772, 0.00532916,
       0.01693275, 0.00733546, 0.01412329, 0.0029418 , 0.00349851,
       0.0038036 , 0.00591866, 0.0024288 , 0.00452373, 0.00296407,
       0.00505475, 0.00382082, 0.01349485, 0.00278734, 0.00808066,
       0.00710367, 0.00390522, 0.00291222, 0.01579389, 0.00230949,
       0.00165816, 0.00349557, 0.00443693, 0.00433772, 0.00436412,
       0.00774706, 0.00322066, 0.01854853, 0.00192736, 0.00658724,
       0.00441913, 0.0062434 , 0.00198642, 0.00330343, 0.00295831,
       0.00385331, 0.01279281, 0.01280461, 0.00402741, 0.01206

In [85]:
dist_j_mask

np.float64(0.007308070497983609)

In [86]:
within_distances

array([0.00695478, 0.00746675, 0.00752416, 0.00722006, 0.00730807])

In [87]:
KK_mask

array([[1.00000000e+000, 7.31289423e-151, 1.67722960e-144, ...,
        3.75227186e-146, 5.10443683e-002, 3.24526852e-002],
       [7.31289423e-151, 1.00000000e+000, 1.52810265e-157, ...,
        1.34688736e-161, 3.65709303e-154, 1.03716490e-149],
       [1.67722960e-144, 1.52810265e-157, 1.00000000e+000, ...,
        1.91351271e-002, 1.46687656e-142, 9.89786704e-144],
       ...,
       [3.75227186e-146, 1.34688736e-161, 1.91351271e-002, ...,
        1.00000000e+000, 5.92613032e-144, 4.24394865e-145],
       [5.10443683e-002, 3.65709303e-154, 1.46687656e-142, ...,
        5.92613032e-144, 1.00000000e+000, 2.45830563e-002],
       [3.24526852e-002, 1.03716490e-149, 9.89786704e-144, ...,
        4.24394865e-145, 2.45830563e-002, 1.00000000e+000]])

In [95]:
gamma = 1 / (2 * median_distance)
K = pairwise_kernels(X, metric='rbf', gamma=gamma)
within_distances = np.zeros(n_classes)
random_state = check_random_state(42)
labels = random_state.randint(n_classes, size=n_samples)
dist = np.zeros((n_samples, n_classes))

In [96]:
sw = np.ones(n_samples)  # Sample weights (n_samples,)

# Step 1: Create the one-hot encoded cluster assignment matrix
Z = np.zeros((len(labels), n_classes))
Z[np.arange(len(labels)), labels] = sw

# Step 2: Compute cluster weights (n_clusters,)
cluster_weights = Z.sum(axis=0)

# Avoid division by zero (handle empty clusters)
# cluster_weights[cluster_weights == 0] = 1

# Step 3: Compute the weighted kernel sum for each cluster
# KZ = K @ Z gives (n_samples, n_clusters)
KZ = K @ Z

# Step 4: Compute the first and second terms
# dist[:] = np.diag(K)[:, np.newaxis] - (2 / cluster_weights) * KZ
dist[:] = - (2 / cluster_weights) * KZ

# Step 5: Update within-cluster distances if needed
# Z.T @ K @ Z gives (n_clusters, n_clusters) weighted kernel sum
ZKZ = Z.T @ K @ Z

within_distances[:] = np.diag(ZKZ) / (cluster_weights ** 2)

# Step 6: Add within-cluster distances to the distance matrix
dist += within_distances

In [97]:
within_distances

array([0.00695478, 0.00746675, 0.00752416, 0.00722006, 0.00730807])

In [98]:
((2 / cluster_weights) * KZ)[:, 4]

array([0.00296766, 0.01462357, 0.00355041, 0.01210312, 0.01395799,
       0.00705427, 0.00503069, 0.00347525, 0.00365044, 0.01248924,
       0.00548069, 0.00490935, 0.01307933, 0.00473925, 0.00925257,
       0.00593841, 0.00254588, 0.01494558, 0.00548426, 0.00606662,
       0.00762713, 0.01241508, 0.00831312, 0.00481035, 0.00220032,
       0.00470827, 0.0053419 , 0.00456536, 0.0076601 , 0.00184823,
       0.00427364, 0.0021476 , 0.01429127, 0.00238772, 0.00532916,
       0.01693275, 0.00733546, 0.01412329, 0.0029418 , 0.00349851,
       0.0038036 , 0.00591866, 0.0024288 , 0.00452373, 0.00296407,
       0.00505475, 0.00382082, 0.01349485, 0.00278734, 0.00808066,
       0.00710367, 0.00390522, 0.00291222, 0.01579389, 0.00230949,
       0.00165816, 0.00349557, 0.00443693, 0.00433772, 0.00436412,
       0.00774706, 0.00322066, 0.01854853, 0.00192736, 0.00658724,
       0.00441913, 0.0062434 , 0.00198642, 0.00330343, 0.00295831,
       0.00385331, 0.01279281, 0.01280461, 0.00402741, 0.01206